In [1]:
# Dependencies
%matplotlib notebook
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
from scipy.stats import linregress
import numpy as np

In [91]:
#Readiing csv files
income_raw = pd.read_csv("../Resources/SAINC1 Personal Income Summary.csv")
expenditure_raw = pd.read_csv("../Resources/SAEXP1__ALL_AREAS_1997_2019.csv")
states_list = pd.read_csv("../Resources/states.csv")

income_raw.head()

,GeoFips,GeoName,LineCode,Description,1929,1930,1931,1932,1933,1934,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,United States,1.0,Personal income (millions of dollars),85151,76394,65531,50162,47114,53967,...,12541995.0,13315478.0,13998383.0,14175503.0,14982715.0,15717140.0,16151881.0,16937582.0,17839255.0,18542262.0
1,0,United States,2.0,Population (persons) 1/,121769000,123075000,124038000,124839000,125580000,126372000,...,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
2,0,United States,3.0,Per capita personal income (dollars) 2/,699,621,528,402,375,427,...,40547.0,42739.0,44605.0,44860.0,47071.0,49019.0,50015.0,52118.0,54606.0,56490.0
3,1000,Alabama,1.0,Personal income (millions of dollars),843.2,697.5,583.7,421.9,435.6,556.3,...,161516.6,167942.8,172804.1,174415.4,180457.7,188711.4,192281.9,199999.8,208752.7,216449.0
4,1000,Alabama,2.0,Population (persons) 1/,2644000,2647000,2649000,2653000,2661000,2685000,...,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0


In [92]:
#Analyse income_raw 
#Create dataframe from 1997 to 2019
#Drop irrelevant columns

yearsdropped = list(range(1929,1997))
yearsdropped
income_1997 = income_raw
for year in yearsdropped:
    income_1997 = income_1997.drop(columns = str(year))

income_df = income_1997.drop(columns = ["GeoFips"])
income_df.head()

,GeoName,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,1.0,Personal income (millions of dollars),7086935.0,7601594.0,8001563.0,8650325.0,9001839.0,9155663.0,9480901.0,...,12541995.0,13315478.0,13998383.0,14175503.0,14982715.0,15717140.0,16151881.0,16937582.0,17839255.0,18542262.0
1,United States,2.0,Population (persons) 1/,272646925.0,275854104.0,279040168.0,282162411.0,284968955.0,287625193.0,290107933.0,...,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
2,United States,3.0,Per capita personal income (dollars) 2/,25993.0,27557.0,28675.0,30657.0,31589.0,31832.0,32681.0,...,40547.0,42739.0,44605.0,44860.0,47071.0,49019.0,50015.0,52118.0,54606.0,56490.0
3,Alabama,1.0,Personal income (millions of dollars),93981.4,99953.3,103453.1,108355.1,112154.9,115191.8,120211.1,...,161516.6,167942.8,172804.1,174415.4,180457.7,188711.4,192281.9,199999.8,208752.7,216449.0
4,Alabama,2.0,Population (persons) 1/,4367935.0,4404701.0,4430141.0,4452173.0,4467634.0,4480089.0,4503491.0,...,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0


In [93]:
#Create population dataframe
population = income_df.loc[income_df["Description"] == "Population (persons) 1/"]
population.set_index(["GeoName"], inplace = True)
population.head()

,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,2004,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
GeoName,,,,,,,,,,,,,,,,,,,,,
United States,2.0,Population (persons) 1/,272646925.0,275854104.0,279040168.0,282162411.0,284968955.0,287625193.0,290107933.0,292805298.0,...,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
Alabama,2.0,Population (persons) 1/,4367935.0,4404701.0,4430141.0,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,...,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0
Alaska,2.0,Population (persons) 1/,612968.0,619932.0,624779.0,627963.0,633714.0,642337.0,648414.0,659286.0,...,713910.0,722128.0,730443.0,737068.0,736283.0,737498.0,741456.0,739700.0,735139.0,731545.0
Arizona,2.0,Population (persons) 1/,4736990.0,4883342.0,5023823.0,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,...,6407172.0,6472643.0,6554978.0,6632764.0,6730413.0,6829676.0,6941072.0,7044008.0,7158024.0,7278717.0
Arkansas,2.0,Population (persons) 1/,2601090.0,2626289.0,2651860.0,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,...,2921964.0,2940667.0,2952164.0,2959400.0,2967392.0,2978048.0,2989918.0,3001345.0,3009733.0,3017804.0


In [94]:
population.loc["Alabama"]

LineCode                             2
Description    Population (persons) 1/
1997                       4.36794e+06
1998                        4.4047e+06
1999                       4.43014e+06
2000                       4.45217e+06
2001                       4.46763e+06
2002                       4.48009e+06
2003                       4.50349e+06
2004                       4.53073e+06
2005                       4.56980e+06
2006                       4.62898e+06
2007                       4.67284e+06
2008                       4.71821e+06
2009                       4.75794e+06
2010                       4.78544e+06
2011                       4.79907e+06
2012                       4.81559e+06
2013                       4.83008e+06
2014                        4.8418e+06
2015                       4.85235e+06
2016                       4.86352e+06
2017                       4.87449e+06
2018                       4.88768e+06
2019                       4.90318e+06
Name: Alabama, dtype: obj

In [98]:
#Analyse expenditure_raw and drop irrelevant headings
expenditure_raw
expenditure_df = expenditure_raw.drop(columns = ["GeoFIPS", "Region", "TableName", "IndustryClassification"])
expenditure_df.set_index(["GeoName", "Description"], inplace = True)
expenditure_df.dropna(axis=0, how='all', inplace=True)
expenditure_df

LineCode  \
GeoName       Description                                                    
United States Personal consumption expenditures                        1.0   
               Goods                                                   2.0   
                Durable goods                                          3.0   
                 Motor vehicles and parts                              4.0   
                 Furnishings and durable household equipment           5.0   
...                                                                    ...   
Far West         Financial services and insurance                     20.0   
                 Other services                                       21.0   
                Final consumption expenditures of nonprofit i...      22.0   
                 Gross output of nonprofit institutions               23.0   
                 Less: Receipts from sales of goods and servi...      24.0   

                                                                                         Unit  \
GeoName       Description                                                                       
United States Personal consumption expenditures                   Millions of current dollars   
               Goods                                              Millions of current dollars   
                Durable goods                                     Millions of current dollars   
                 Motor vehicles and parts                         Millions of current dollars   
                 Furnishings and durable household equipment      Millions of current dollars   
...                                                                                       ...   
Far West         Financial services and insurance                 Millions of current dollars   
                 Other services                                   Millions of current dollars   
                Final consumption expenditures of nonprofit i...  Millions of current dollars   
                 Gross output of nonprofit institutions           Millions of current dollars   
                 Less: Receipts from sales of goods and servi...  Millions of current dollars   

                                                                       1997  \
GeoName       Description                                                     
United States Personal consumption expenditures                   5534091.8   
               Goods                                              2003807.2   
                Durable goods                                      715529.6   
                 Motor vehicles and parts                          293082.5   
                 Furnishings and durable household equipment       160490.2   
...                                                                     ...   
Far West         Financial services and insurance                   83502.2   
                 Other services                                     73099.5   
                Final consumption expenditures of nonprofit i...    15583.2   
                 Gross output of nonprofit institutions             79226.1   
                 Less: Receipts from sales of goods and servi...    63642.9   

                                                                       1998  \
GeoName       Description                                                     
United States Personal consumption expenditures                   5874285.0   
               Goods                                              2105476.2   
                Durable goods                                      779293.4   
                 Motor vehicles and parts                          320204.7   
                 Furnishings and durable household equipment       173579.1   
...                                                                     ...   
Far West         Financial services and insurance                   91262.2   
                 Other services                   

In [99]:
expenditure_df.loc[["Florida", "Personal consumption expenditures"]]
x = expenditure_df.index.values
list(x[0])
expenditure_df.loc[("Florida", "Personal consumption expenditures")]["1997"]

314431.2

In [100]:
#Change expenditure dataframe to be in per capital values
# fill NaN in expendinture with 0
expenditure_df= expenditure_df.fillna(0)

#Creating empty description series in database
expenditure_capita = expenditure_df[['LineCode', 'Unit']]

#expenditure_capita["Description"] = ''
expenditure_capita

LineCode  \
GeoName       Description                                                    
United States Personal consumption expenditures                        1.0   
               Goods                                                   2.0   
                Durable goods                                          3.0   
                 Motor vehicles and parts                              4.0   
                 Furnishings and durable household equipment           5.0   
...                                                                    ...   
Far West         Financial services and insurance                     20.0   
                 Other services                                       21.0   
                Final consumption expenditures of nonprofit i...      22.0   
                 Gross output of nonprofit institutions               23.0   
                 Less: Receipts from sales of goods and servi...      24.0   

                                                                                         Unit  
GeoName       Description                                                                      
United States Personal consumption expenditures                   Millions of current dollars  
               Goods                                              Millions of current dollars  
                Durable goods                                     Millions of current dollars  
                 Motor vehicles and parts                         Millions of current dollars  
                 Furnishings and durable household equipment      Millions of current dollars  
...                                                                                       ...  
Far West         Financial services and insurance                 Millions of current dollars  
                 Other services                                   Millions of current dollars  
                Final consumption expenditures of nonprofit i...  Millions of current dollars  
                 Gross output of nonprofit institutions           Millions of current dollars  
                 Less: Receipts from sales of goods and servi...  Millions of current dollars  

[1440 rows x 2 columns]

In [104]:
#Changing Description series to report correct units i.e. per Capita
for year in list(range(1997,2020)):
    expenditure_capita[str(year)]=None   
    
    
#expenditure_capita.loc[("Florida", "Personal consumption expenditures")]["1997"] = expenditure_df.loc[("Florida", "Personal consumption expenditures")]["1997"]

    for indx in expenditure_df.index.values:
              #expenditure_capita.iloc[i,2] = f'{expenditure_df.iloc[i,2]} (USD/Capita)'
        expenditure_capita[str(year)].loc[(indx)] = expenditure_df.loc[(indx)][str(year)] * 1000000 / population.loc[indx[0]][str(year)]
expenditure_capita

C:\Users\dell\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


LineCode  \
GeoName       Description                                                    
United States Personal consumption expenditures                        1.0   
               Goods                                                   2.0   
                Durable goods                                          3.0   
                 Motor vehicles and parts                              4.0   
                 Furnishings and durable household equipment           5.0   
...                                                                    ...   
Far West         Financial services and insurance                     20.0   
                 Other services                                       21.0   
                Final consumption expenditures of nonprofit i...      22.0   
                 Gross output of nonprofit institutions               23.0   
                 Less: Receipts from sales of goods and servi...      24.0   

                                                                                         Unit  \
GeoName       Description                                                                       
United States Personal consumption expenditures                   Millions of current dollars   
               Goods                                              Millions of current dollars   
                Durable goods                                     Millions of current dollars   
                 Motor vehicles and parts                         Millions of current dollars   
                 Furnishings and durable household equipment      Millions of current dollars   
...                                                                                       ...   
Far West         Financial services and insurance                 Millions of current dollars   
                 Other services                                   Millions of current dollars   
                Final consumption expenditures of nonprofit i...  Millions of current dollars   
                 Gross output of nonprofit institutions           Millions of current dollars   
                 Less: Receipts from sales of goods and servi...  Millions of current dollars   

                                                                     1997  \
GeoName       Description                                                   
United States Personal consumption expenditures                   20297.6   
               Goods                                              7349.46   
                Durable goods                                     2624.38   
                 Motor vehicles and parts                         1074.95   
                 Furnishings and durable household equipment      588.637   
...                                                                   ...   
Far West         Financial services and insurance                 1853.39   
                 Other services                                   1622.49   
                Final consumption expenditures of nonprofit i...   345.88   
                 Gross output of nonprofit institutions           1758.48   
                 Less: Receipts from sales of goods and servi...   1412.6   

                                                                     1998  \
GeoName       Description                                                   
United States Personal consumption expenditures                   21294.9   
               Goods                                              7632.57   
                Durable goods                                     2825.02   
                 Motor vehicles and parts                         1160.78   
                 Furnishings and durable household equipment      629.242   
...                                                                   ...   
Far West         Financial services and insurance                 1992.71   
                 Other services                                   1729.52   
                F

In [103]:
population.loc[indx[0]][str(year)]

56572426.0

In [ ]:
    
# Creating empty dataframe in order to set headings
year =  list(range(1997,2020))
for year in year:
    expenditure_capita[str(year)] = ''
expenditure_capita.head()

In [ ]:
expenditure_df

In [ ]:
#Convert Millions of Dollars in expenditure_df to USD per Capita


for i in range(len())
    expenditure_capita.iloc[i,j+3] = expenditure_df.iloc[i,j+3] * 1000000 / 

In [ ]:
#Analyze states
states_list.head()

In [ ]:
#merge income and expenditure dataframes and rename Geoname to State
income_expend = pd.merge( income_df, expenditure_df, how='outer')
income_expend = income_expend.rename(columns = {'GeoName':'State'})
income_expend.head()

In [ ]:
#merge dataframes with states list to get rid of all rows that are not states
combined_df = pd.merge( states_list, income_expend, how='left', on="State")
combined_df.head(20)

In [ ]:
income_expend = income_expend.sort_values(['GeoName','LineCode'])
income_expend.loc[(income_expend["GeoName"]=="Alabama")\
                  & (income_expend["Description"]=="Personal income (millions of dollars)")\
                 | (income_expend["Description"]=="Durable goods)")]#.iloc[:, 3:8]